## Mount the Drive, and Change to Google Drive Folder

In [1]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

# %cd /content/drive/MyDrive/MSc.-Dissertations/1/Files
# %ls

## Import Libraries

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, utils, losses
from keras.wrappers import scikit_learn
from keras.models import *
from keras.layers import *
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import np_utils

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

import random
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## Count the Number of Files, and Take Random Samples from the Image Files

In [3]:
# !ls street_view
# count how many files and write the filenames into a file
# !ls street_view -1 | wc -l 
# !ls street_view/*.jpg > flist.txt
flist = list(pd.read_csv('flist.txt', header = None)[0])

# Set seed so sample is reproducible 
# random.seed(99)  # set this to an integer value!!!
# nsamp = 100
# flist_sub = random.sample(flist, nsamp)
# flist = flist_sub

# print(flist)

## Overview of the `properties` Dataset

Read the `properties` dataset first, and make sure that `property type` is a categorical variable.

In [4]:
properties = pd.read_csv('properties.csv')
properties.propertyType = properties.propertyType.astype('category')
properties.head()

,Unnamed: 0,address,propertyType,bedrooms,detailUrl,location_lat,location_lng,property_id
0,0,"12, Gorsey Brigg, Dronfield Woodhouse, Dronfie...",Terraced,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29986,-1.49446,60d9dd15-c5a0-4d9c-a341-a1d47add49d5
1,0,"5, Highgate Lane, Dronfield, Derbyshire S18 1UB",Detached,4.0,https://www.rightmove.co.uk/house-prices/detai...,53.29135,-1.45975,4a586e80-181a-4b82-b5c3-2d789436bb14
2,0,"125, Gosforth Lane, Dronfield, Derbyshire S18 1RB",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29763,-1.47573,93680b6c-237e-44d3-8f40-959a14b80cad
3,0,"80, Shakespeare Crescent, Dronfield, Derbyshir...",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29259,-1.45644,5d49758b-f148-4d06-bbae-3eb23f5c68fb
4,0,"21, Gainsborough Road, Dronfield, Derbyshire S...",Detached,NaN,https://www.rightmove.co.uk/house-prices/detai...,53.29740,-1.48503,4645f5eb-de7c-474f-8d7e-b59fa8c55f19


Basic information of the dataset is shown as follows.

In [5]:
properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17550 entries, 0 to 17549
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Unnamed: 0    17550 non-null  int64   
 1   address       17550 non-null  object  
 2   propertyType  17550 non-null  category
 3   bedrooms      11505 non-null  float64 
 4   detailUrl     17550 non-null  object  
 5   location_lat  17550 non-null  float64 
 6   location_lng  17550 non-null  float64 
 7   property_id   17550 non-null  object  
dtypes: category(1), float64(3), int64(1), object(3)
memory usage: 977.2+ KB


Descriptive statistics of continuous variables are shown as follows.

In [6]:
properties.describe()

,Unnamed: 0,bedrooms,location_lat,location_lng
count,17550.0,11505.000000,17550.000000,17550.000000
mean,0.0,2.871186,52.912264,-2.330492
std,0.0,1.010339,1.833830,1.262468
min,0.0,0.000000,50.617080,-4.268950
25%,0.0,2.000000,51.232830,-3.067290
50%,0.0,3.000000,53.095885,-2.658955
75%,0.0,3.000000,53.846760,-1.712750
max,0.0,6.000000,55.910540,0.719990


Frequencies of each level of the variable `property type` are obtained as follows.

In [7]:
properties.propertyType.value_counts()

Detached         4134
Semi-Detached    4056
Unknown          3900
Terraced         3666
Flat             1794
Name: propertyType, dtype: int64

## A Subset of the `properties` Dataset

As random samples of images have been obtained previously, a subset of the whole `properties` dataset could hence be formulated by selecting the rows of the whole `properties` dataset corresponding to the selected samples.

In [8]:
flist_id = list(map(lambda string: string[16 : -4], flist))
properties_sub = pd.DataFrame(properties.loc[properties['property_id'].isin(flist_id)])
properties_sub = properties_sub.drop_duplicates(['location_lat', 'location_lng'])
flist_id = list(properties_sub.property_id)
flist_new = []
temp = list(map(lambda x: flist_new.append(f'street_view/gsv_{x}.jpg'), flist_id))

The original data should be splitted into training and testing sets, and the testing set contains 30% of the original data.

In [9]:
Img_array_train, Img_array_test, propertyType_train, propertyType_test = train_test_split(
    np.array(list(map(lambda x: np.array(Image.open(x)), flist_new))), # RGB values in each pixel
    np.array(properties_sub.propertyType), # property types
    test_size = 0.3)

For categorical variables, one-hot encoder is introduced.

In [10]:
dummy_propertyType_train = pd.get_dummies(propertyType_train)
dummy_propertyType_test = pd.get_dummies(propertyType_test)
propertyType_test_fac = np.argmax(np.array(dummy_propertyType_test), axis = 1) 

## Multi-Class Classification Using Neural Network

### Multi-Layer Perceptron (MLP) model

In [11]:
def mlp(output_dim):

    '''
    Creates a multi-layer perceptron neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Flatten())
    # model.add(Dense(10, activation = tf.nn.leaky_relu))
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [12]:
MLP = mlp(dummy_propertyType_train.shape[1])
MLP.fit(Img_array_train, dummy_propertyType_train, epochs = 20, batch_size = 30)

Epoch 1/20
10/10 [==============================] - 2s 79ms/step - loss: 103.2740 - accuracy: 0.2426
Epoch 2/20
10/10 [==============================] - 1s 78ms/step - loss: 100.0788 - accuracy: 0.2206
Epoch 3/20
10/10 [==============================] - 1s 73ms/step - loss: 71.3005 - accuracy: 0.3051
Epoch 4/20
10/10 [==============================] - 1s 75ms/step - loss: 40.7091 - accuracy: 0.2684
Epoch 5/20
10/10 [==============================] - 1s 63ms/step - loss: 21.9799 - accuracy: 0.3787
Epoch 6/20
10/10 [==============================] - 1s 68ms/step - loss: 35.2445 - accuracy: 0.3382
Epoch 7/20
10/10 [==============================] - 1s 70ms/step - loss: 22.4040 - accuracy: 0.3566
Epoch 8/20
10/10 [==============================] - 1s 78ms/step - loss: 19.7575 - accuracy: 0.4044
Epoch 9/20
10/10 [==============================] - 1s 69ms/step - loss: 18.7245 - accuracy: 0.4044
Epoch 10/20
10/10 [==============================] - 1s 83ms/step - loss: 13.2576 - accuracy: 0.40

In [13]:
propertyType_test_pred = np.argmax(MLP.predict(Img_array_test), axis = 1) 

4/4 [==============================] - 0s 26ms/step


In [14]:
confusion_matrix(propertyType_test_fac, propertyType_test_pred)

array([[ 0,  0,  2, 21,  1],
       [ 0,  0,  0, 10,  1],
       [ 0,  0,  1, 28,  7],
       [ 0,  0,  1, 22,  2],
       [ 0,  0,  4,  8,  9]], dtype=int64)

## Convolutional Neural Network

In [15]:
def cnn(output_dim):

    '''
    Creates a convolutional neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Conv2D(5, 2, padding = 'same', activation = tf.nn.leaky_relu))
    model.add(MaxPooling2D())   
    model.add(Flatten())
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [16]:
CNN = cnn(dummy_propertyType_train.shape[1])
CNN.fit(Img_array_train, dummy_propertyType_train, epochs = 15, batch_size = 50)

Epoch 1/15
6/6 [==============================] - 4s 543ms/step - loss: 21.2211 - accuracy: 0.2390
Epoch 2/15
6/6 [==============================] - 3s 584ms/step - loss: 18.2231 - accuracy: 0.1397
Epoch 3/15
6/6 [==============================] - 3s 543ms/step - loss: 8.4986 - accuracy: 0.2904
Epoch 4/15
6/6 [==============================] - 3s 489ms/step - loss: 3.7932 - accuracy: 0.3051
Epoch 5/15
6/6 [==============================] - 3s 493ms/step - loss: 2.4672 - accuracy: 0.3713
Epoch 6/15
6/6 [==============================] - 3s 575ms/step - loss: 2.1491 - accuracy: 0.4154
Epoch 7/15
6/6 [==============================] - 3s 524ms/step - loss: 1.6174 - accuracy: 0.4485
Epoch 8/15
6/6 [==============================] - 3s 558ms/step - loss: 1.4174 - accuracy: 0.5441
Epoch 9/15
6/6 [==============================] - 3s 564ms/step - loss: 0.9123 - accuracy: 0.6434
Epoch 10/15
6/6 [==============================] - 3s 518ms/step - loss: 0.8672 - accuracy: 0.6471
Epoch 11/15
6/6 [

In [17]:
propertyType_test_pred = np.argmax(CNN.predict(Img_array_test), axis = 1) 

4/4 [==============================] - 1s 228ms/step


In [18]:
confusion_matrix(propertyType_test_fac, propertyType_test_pred)

array([[10,  3,  4,  6,  1],
       [ 3,  4,  0,  3,  1],
       [10,  5,  6,  4, 11],
       [ 5,  4,  4,  5,  7],
       [ 4,  3,  3,  2,  9]], dtype=int64)